In [1]:
using Printf
using Random
using LinearAlgebra
using PlotlyJS
rng = MersenneTwister(18);

WebIO._IJuliaInit()

In [2]:
n = 3 # In 3D

Q = rand(n,n)
Q,R = qr(Q)
Λ = diagm(0 => Float32[2.0^(-i) for i=0:n-1])
A = Q * Λ * Q'
# The e-vectors are the columns of Q.

Diagonal(Λ)

3×3 Diagonal{Float32, Vector{Float32}}:
 1.0   ⋅    ⋅ 
  ⋅   0.5   ⋅ 
  ⋅    ⋅   0.25

In [3]:
μ0 = Λ[1,1]
μ = μ0 * 0.8
# Our initial approximation of an eigenvalue of interest

println("Exact answer: ", μ0)
println("Initial guess: ", μ)

qk = rand(rng, n)
qk /= norm(qk)

# Estimating C fit
function C_estimate(A, μ0, μ_, qk_)
    μ = μ_
    qk = qk_
    error0 = 0
    for k=1:2
        zk = (A - μ * I) \ qk
        qk = zk/norm(zk)      # Normalize e-vector
        μ = dot(qk, A * qk)   # E-value approximation
        error0 = abs(μ-μ0)
    end
    zk = (A - μ * I) \ qk
    qk = zk/norm(zk)      # Normalize e-vector
    μ = dot(qk, A * qk)   # E-value approximation
    error1 = abs(μ-μ0)
    ϵ = sqrt(error1/error0)
    return error0 / ϵ
end

c_fit = C_estimate(A, μ0, μ, qk)
error_old = -1

for k=1:5
    zk = (A - μ * I) \ qk
    qk = zk/norm(zk)      # Normalize e-vector
    μ = dot(qk, A * qk)   # E-value approximation
    @printf "Iteration: %1d | e-value: %10.8f | error: %7.1e " k μ abs(μ-μ0)
    if k>1
        @printf " (estim. %7.1e)\n" c_fit*(error_old/c_fit)^3.
    else
        @printf "\n"
    end
    error_old = abs(μ-μ0)
end

Exact answer: 1.0
Initial guess: 0.8
Iteration: 1 | e-value: 0.99542029 | error: 4.6e-03 
Iteration: 2 | e-value: 0.99999979 | error: 2.1e-07  (estim. 4.9e-03)
Iteration: 3 | e-value: 1.00000000 | error: 4.4e-16  (estim. 4.4e-16)
Iteration: 4 | e-value: 1.00000000 | error: 2.2e-16  (estim. 4.5e-42)
Iteration: 5 | e-value: 1.00000000 | error: 0.0e+00  (estim. 5.6e-43)


In [4]:
# Let's plot the Rayleigh quotient to observe that grad r = 0
# at all e-values.

N = 32
u = LinRange(0, 2π, N)
v = LinRange(0, π, N)

x = cos.(u) * sin.(v)'
y = sin.(u) * sin.(v)'
z = repeat(cos.(v)',outer=[N, 1])

r = zeros(N,N)
for i=1:N, j=1:N
    # The point q is on the surface of the unit sphere
    q = [x[i,j] y[i,j] z[i,j]]
    # Rayleigh-quotient at this point
    r[i,j] = dot(q, A * transpose(q))
end

P = 1.1*vcat(Q',-Q') # Increase the radius a bit

6×3 Matrix{Float64}:
 -0.803312  -0.512639  -0.549446
 -0.456145  -0.306513   0.952881
 -0.597178   0.923716   0.0112619
  0.803312   0.512639   0.549446
  0.456145   0.306513  -0.952881
  0.597178  -0.923716  -0.0112619

In [5]:
t = scatter3d(x=P[:,1], y=P[:,2], z=P[:,3],
    mode="markers", marker_size=10)

s = surface(x=x, y=y, z=z, surfacecolor=r)

l = Layout(title="Rayleigh quotient", 
    aspectratio=attr(x=1, y=1, z=1),
    scene_dragmode="orbit")

plot([s, t], l)

data: [
  "surface with fields surfacecolor, type, x, y, and z",
  "scatter3d with fields marker, mode, type, x, y, and z"
]

layout: "layout with fields aspectratio, margin, scene, template, and title"